Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 1a Test Majority Ensembling - Roberta Only

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os

In [2]:
# Loading up all the predictions data

n_rnds = 3
original_df = pd.read_csv('../Datasets/Subtask_1a/test.tsv', sep='\t')
models = ['roberta-large']
n_models = len(models)

epoch_string = '../2nd_try_15_epochs_large_model/saved_test_result_1a'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())



In [3]:
# Retrieving all the predictions from the 
list_of_df = []

for model in models: 
    specific_model_row = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_test_result.tsv'
            
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)
    
    list_of_df.append(specific_model_row)

In [4]:
# TAKING THE MAJORITY OF DATA

majority_original_tweet_id_list = []
majority_original_sentence_list = []
majority_original_numbered_label_list = []

majority_predicted_number_results = []
majority_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['tweet_id']
    original_sentence = row['tweet']
    
    # transferring the labels over to final list
    majority_original_tweet_id_list.append(original_tweet_id)
    majority_original_sentence_list.append(original_sentence)
    
    specific_row_results = []
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):
            # print(list_of_df[model_num][rnd_num])
            row = list_of_df[model_num][rnd_num].loc[(list_of_df[model_num][rnd_num]['tweet_id'] == original_tweet_id)]

            prediction = labels_to_ids[row['class'].values[0]]
            specific_row_results.append(prediction)

    specific_row_results = Counter(specific_row_results)
    specific_row_results = specific_row_results.most_common(1)[0][0]
    majority_predicted_results.append(ids_to_labels[specific_row_results])
    

KeyError: 'tweet_id'

In [ ]:
# Saving it as a dataframe
formatted_majority_prediction_data = pd.DataFrame(zip(majority_original_tweet_id_list, majority_original_sentence_list, majority_predicted_results), columns=['id', 'text', 'class'])
formatted_majority_prediction_data

,id,text,Claim,Stance
0,1307558525371965442,@narendramodi @rajnathsingh Student ka bhi soa...,school closures,NONE
1,1247739239879467009,—échale un vistazo a esto… … a fair piece on...,stay at home orders,NONE
2,1242046510155653125,Why do think skilling women and girls is impor...,stay at home orders,NONE
3,1358446499949084675,To reduce the risk of the virus spreading as e...,school closures,FAVOR
4,1249740062775902208,I speak for a great many people when i say WE ...,stay at home orders,AGAINST
...,...,...,...,...
9950,1242516037628813314,StayAtHomeSaveLives 21daysLockdown StayAtHome ...,stay at home orders,FAVOR
9951,1242746919933415424,If this is true this is heartbreaking StayAtHo...,stay at home orders,FAVOR
9952,1276638598813679617,"855 Sunset Cove Dr, Winter Haven, FL 33880 3 B...",stay at home orders,NONE
9953,1243504288661270528,StayAtHomeSaveLives StayHomeStaySafe StayHome ...,stay at home orders,FAVOR


In [ ]:
# Saving it as a tsv file
os.makedirs('../2nd_try_15_epochs_large_model/final_test_result', exist_ok=True)
formatted_majority_prediction_data.to_csv('../2nd_try_15_epochs_large_model/final_test_result/formatted_majority_data.tsv', sep='\t', index=False)